## Algoritmo para calcular automáticamente las diferencias finitas de $f(x)$ a ordenes más altos:

Editar las rutinas presentadas en esta clase para calcular la _derivada hacia adelante_ a orden más alto, pero en esta ocasión para el operador _derivada simetrizada_.
* No olvide hacer el test de la rutina, similar a lo que se hizo en este notebook para el $cos$.

In [1]:
using SymPy

La rutinas mostradas a continuación son muy similares a las vistas en clase, sólo que ahora se utiliza el Arc_Sen de (x/2) en lugar de log(x+1), debido a que:

$$\left. \frac { d }{ d\xi  }  \right| _{ \xi =x }=\frac { 1 }{ h } { asinh} \left( \frac { \delta _{ h } }{ 2 }  \right) $$

También se cambia la función finite_difference_forward(f) por finite_difference_simetric(f), lo que implica que ahora se utiliza:

$$(x,h)->(f(x+h,h)-f(x-h,h))$$

In [2]:
#Para obtener los coeficientes de Taylor se utiliza SymPy para considerar a x y a h como variables simbólicas y posteriormente se utiliza N, para que las funciones de los coeficientes no arrojen resultados de tipo simbólico. 

function coeff_Taylor_Arc_Sen(i) 
    x=Sym("x");
    h=Sym("h");
    N(subs(diff(asinh(x/2),x,i)/factorial(Float64(i)),x=>0))
end;

function coeff_Taylor_Arc_Sen_rational(i)
    x=Sym("x");
    h=Sym("h");
    N(subs(diff(asinh(x/2),x,i)/factorial(i),x=>0))
end;

function finite_difference_simetric(f)
    return (x,h)->(f(x+h,h)-f(x-h,h))
end;   

Las siguientes funciones que calculan las derivadas hasta un determinado orden son muy similares, sólo que para el primero se utiliza la función coeff_Taylor_Arc_Sen(i), mientras que para el segundo se utiliza la función coeff_Taylor_Arc_Sen_rational(i).

Se utiliza un ciclo for que avanza desde el orden 1 hasta el orden escogido, para cada vuelta del ciclo se calcula la derivada simétrica, que posteriormente se multiplica por el valor obtenido con la función que calcula los coeficientes de Taylor con la i correspondiente a la vuelta del ciclo; el resultado obtenido en cada ciclo se va sumando a la derivada que se inicializa como cero. Finalmente la derivada obtenida cuando el ciclo for finaliza se divide entre h.

In [3]:
function Derivative_order(f,x,h,order)
    g(x,h)=f(x)
    der=0
    for i in range(1,order)
        g=finite_difference_simetric(g)
        der=der+coeff_Taylor_Arc_Sen(i)*g(x,h)
    end
    der/h
end;

function Derivative_order_rational_coefficients(f,x,h,order)
    g(x,h)=f(x)
    der=0
    for i in range(1,order)
        g=finite_difference_simetric(g)
        der=der+coeff_Taylor_Arc_Sen_rational(i)*g(x,h)
    end
    der/h
end;

### Test

Se prueba con la función coseno, en x=10.0, con un valor de h=0.01, considerando un orden 6.

In [4]:
Derivative_order(cos,10.0,0.01,6)

0.5440211108893371

In [5]:
Derivative_order_rational_coefficients(cos,10.0,0.01,6)

0.5440211108893371

In [7]:
-sin(10.0)

0.5440211108893698

In [8]:
f=SymFunction("f");

Para conocer el orden del error de este método, expandimos la derivada numérica simétrica en serie de Taylor:

In [9]:
aprox=Derivative_order_rational_coefficients(SymFunction("f"),Sym("x"),Sym("h"),2)

  f(-h + x)   f(h + x)
- --------- + --------
      2          2    
----------------------
          h           

In [10]:
series(aprox,Sym("h"),0,5)

                             /   3           \|            /   5           \| 
                           2 |  d            ||          4 |  d            || 
                          h *|------(f(xi_1))||         h *|------(f(xi_1))|| 
                             |     3         ||            |     5         || 
/  d           \|            \dxi_1          /|xi_1=x      \dxi_1          /|x
|-----(f(xi_1))||       + --------------------------- + ----------------------
\dxi_1         /|xi_1=x                6                            120       

             
             
             
             
i_1=x    / 5\
----- + O\h /
             

Si se toman más términos de la serie:

In [11]:
aprox=Derivative_order_rational_coefficients(SymFunction("f"),Sym("x"),Sym("h"),5)

  3*f(-5*h + x)   25*f(-3*h + x)   75*f(-h + x)   75*f(h + x)   25*f(3*h + x) 
- ------------- + -------------- - ------------ + ----------- - ------------- 
       1280            768             128            128            768      
------------------------------------------------------------------------------
                                             h                                

  3*f(5*h + x)
+ ------------
      1280    
--------------
              

In [12]:
series(aprox,Sym("h"),0,6)

/  d           \|          / 6\
|-----(f(xi_1))||       + O\h /
\dxi_1         /|xi_1=x        

Se observa que el orden del error obtenido para la aproximación de la derivada de orden superior uitilizando coeficientes racionales, en el punto 0, hasta un orden 6, es de orden 6.